# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [25]:
import pandas as pd
import pymongo
from pymongo import MongoClient
import json

In [26]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

In [27]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

In [28]:
# Connexion à MongoDB
client = MongoClient('localhost', 27017)  # Vous pouvez modifier l'URL si vous utilisez MongoDB Atlas
db = client['youtube_db']

### Netoyer les données

In [29]:

with open('./data/US_category_id.json') as f:
    categories_data = json.load(f)



### Importer les données

In [30]:

videos = df_youtube.to_dict(orient='records')
videos_collection = db['videos']
videos_collection.insert_many(videos)

InsertManyResult([ObjectId('678e2ae623f53d30f32ae39c'), ObjectId('678e2ae623f53d30f32ae39d'), ObjectId('678e2ae623f53d30f32ae39e'), ObjectId('678e2ae623f53d30f32ae39f'), ObjectId('678e2ae623f53d30f32ae3a0'), ObjectId('678e2ae623f53d30f32ae3a1'), ObjectId('678e2ae623f53d30f32ae3a2'), ObjectId('678e2ae623f53d30f32ae3a3'), ObjectId('678e2ae623f53d30f32ae3a4'), ObjectId('678e2ae623f53d30f32ae3a5'), ObjectId('678e2ae623f53d30f32ae3a6'), ObjectId('678e2ae623f53d30f32ae3a7'), ObjectId('678e2ae623f53d30f32ae3a8'), ObjectId('678e2ae623f53d30f32ae3a9'), ObjectId('678e2ae623f53d30f32ae3aa'), ObjectId('678e2ae623f53d30f32ae3ab'), ObjectId('678e2ae623f53d30f32ae3ac'), ObjectId('678e2ae623f53d30f32ae3ad'), ObjectId('678e2ae623f53d30f32ae3ae'), ObjectId('678e2ae623f53d30f32ae3af'), ObjectId('678e2ae623f53d30f32ae3b0'), ObjectId('678e2ae623f53d30f32ae3b1'), ObjectId('678e2ae623f53d30f32ae3b2'), ObjectId('678e2ae623f53d30f32ae3b3'), ObjectId('678e2ae623f53d30f32ae3b4'), ObjectId('678e2ae623f53d30f32ae3

## Question 1  

In [31]:

apple_videos = videos_collection.find({'channel_title': 'Apple'})

for video in apple_videos:
    print(f"Title: {video['title']}, Views: {video['views']}")

Title: iPhone X — Introducing iPhone X — Apple, Views: 7860119
Title: iPhone X — Introducing iPhone X — Apple, Views: 12200526
Title: iPhone X — Introducing iPhone X — Apple, Views: 15731493
Title: iPhone X — Introducing iPhone X — Apple, Views: 18082737
Title: iPhone X — Introducing iPhone X — Apple, Views: 19707391
Title: Apple Watch Series 3 + Apple Music — Roll — Apple, Views: 782543
Title: Apple Watch Series 3 + Apple Music — Roll — Apple, Views: 3987580
Title: Apple Watch Series 3 + Apple Music — Roll — Apple, Views: 4112187
Title: iPhone 8 Plus — Portraits of Her  — Apple, Views: 117564
Title: iPhone 8 Plus — Portraits of Her  — Apple, Views: 163024
Title: iPhone 8 Plus — Portraits of Her  — Apple, Views: 183325
Title: iPhone X — Introducing iPhone X — Apple, Views: 7860119
Title: iPhone X — Introducing iPhone X — Apple, Views: 12200526
Title: iPhone X — Introducing iPhone X — Apple, Views: 15731493
Title: iPhone X — Introducing iPhone X — Apple, Views: 18082737
Title: iPhone X 

## Question 2

In [32]:
distinct_categories = videos_collection.distinct('category_id')
print(f"Nombre de catégories différentes : {len(distinct_categories)}")

Nombre de catégories différentes : 16


## Question 3

In [38]:

for video in videos_collection.find():
    if isinstance(video['tags'], str):  
        tags_list = video['tags'].split('|') if video['tags'] else []  
    elif isinstance(video['tags'], list):  
        tags_list = video['tags']
    else:
        tags_list = []  

    videos_collection.update_one(
        {'_id': video['_id']},
        {'$set': {'tags': tags_list}}
    )

print("tag mis à jours.")


tag mis à jours.


## Question 4

In [34]:
most_viewed_videos = videos_collection.find().sort('views', -1).limit(5)

for video in most_viewed_videos:
    print(f"Title: {video['title']}, Views: {video['views']}")

Title: BTS (방탄소년단) 'DNA' Official MV, Views: 41500672
Title: BTS (방탄소년단) 'DNA' Official MV, Views: 41500672
Title: BTS (방탄소년단) 'DNA' Official MV, Views: 41500672
Title: BTS (방탄소년단) 'DNA' Official MV, Views: 38013692
Title: BTS (방탄소년단) 'DNA' Official MV, Views: 38013692


## Question 5

In [35]:
pipeline = [
    {
        '$group': {
            '_id': '$category_id',
            'average_views': {'$avg': '$views'}
        }
    }
]

average_views_per_category = videos_collection.aggregate(pipeline)

for category in average_views_per_category:
    print(f"Category ID: {category['_id']}, Average Views: {category['average_views']}")

Category ID: 15, Average Views: 651404.3879310344
Category ID: 22, Average Views: 971532.7573696146
Category ID: 25, Average Views: 540955.5569823436
Category ID: 27, Average Views: 547582.4700598803
Category ID: 10, Average Views: 1176553.612
Category ID: 28, Average Views: 924730.548828125
Category ID: 1, Average Views: 1039472.6560846561
Category ID: 24, Average Views: 1154868.4397251718
Category ID: 26, Average Views: 537665.1806674339
Category ID: 29, Average Views: 1110334.2142857143
Category ID: 17, Average Views: 728434.3243902439
Category ID: 2, Average Views: 607693.8706896552
Category ID: 19, Average Views: 464041.0833333333
Category ID: 20, Average Views: 681081.8170731707
Category ID: 43, Average Views: 8492.5
Category ID: 23, Average Views: 1240073.294039735


## Question 6 

In [36]:
pipeline = [
    {
        '$group': {
            '_id': '$channel_title',
            'average_likes': {'$avg': '$likes'}
        }
    },
    {
        '$sort': {'average_likes': -1}
    },
    {
        '$limit': 5
    }
]

channels_with_highest_likes = videos_collection.aggregate(pipeline)

for channel in channels_with_highest_likes:
    print(f"Channel: {channel['_id']}, Average Likes: {channel['average_likes']}")

Channel: ZaynVEVO, Average Likes: 1431683.0
Channel: ibighit, Average Likes: 1371766.25
Channel: melanie martinez, Average Likes: 911871.5
Channel: BETNetworks, Average Likes: 769687.625
Channel: jypentertainment, Average Likes: 758826.6666666666
